In [ ]:
import hebi
from time import sleep

lookup = hebi.Lookup()
# Wait 2 seconds for the module list to populate
sleep(2.0)

family_name = "Arm"
module_names = ["J1_base", "J2_shoulder", "J3_elbow", "J4_wrist"]

group = lookup.get_group_from_names([family_name], module_names)

if group is None:
    print('Group not found: Did you forget to set the module family and name above?')
    exit(1)

try:
    model = hebi.robot_model.import_from_hrdf("arm-model.hrdf")
except:
    print("Could not load HRDF.")
    exit(1)
print(group)


def feedback_handler(group_fbk):
    angles = group_fbk.position
    transform = model.get_end_effector(angles)


group.add_feedback_handler(feedback_handler)
group.feedback_frequency = 10.0

In [ ]:
################################################################
# Get position feedback from robot to use as initial conditions for local optimization.
################################################################
transform = None
group_fbk = hebi.GroupFeedback(group.size)
if group.get_next_feedback(reuse_fbk=group_fbk) is None:
    print("Couldn't get feedback.")
    exit(1)

def feedback_handler(group_fbk):
    global transform
    angles = group_fbk.position
    transform = model.get_end_effector(angles)

group.add_feedback_handler(feedback_handler)
group.feedback_frequency = 10.0

def get_position_feedback():
    # Note: user should check if the positions are appropriate for initial conditions.
    initial_joint_angles = group_fbk.position
    return initial_joint_angles

In [ ]:
################################################################
# Get IK Solution with one objective
################################################################

# Just one objective:
# Note: this is a numerical optimization and can be significantly affected by initial conditions (seed joint angles)
def calc_ik_target_angles(target):
    ee_pos_objective = hebi.robot_model.endeffector_position_objective(target)
    ik_result_joint_angles = model.solve_inverse_kinematics(get_position_feedback(), ee_pos_objective)

    print('Target position: {0}'.format(target))
    print('IK joint angles: {0}'.format(ik_result_joint_angles))
    print('FK of IK joint angles: {0}'.format(model.get_end_effector(ik_result_joint_angles)[0:3, 3]))
    return ik_result_joint_angles

In [ ]:
################################################################
# Send commands to the physical robot
################################################################

# Move the arm
# Note: you could use the Hebi Trajectory API to do this smoothly
def send_group_command(ik_result_joint_angles):
    group_command = hebi.GroupCommand(group.size)
    group_command.position = ik_result_joint_angles
    group.send_command(group_command)
    

In [ ]:
def get_current_position():
    global transform
    #print('x,y,z: {0}, {1}, {2}'.format(transform[0, 3], transform[1, 3], transform[2, 3]))
    return (transform)

In [ ]:
import numpy as np
for _ in range(1000):
    #print(get_current_position())
    
    posArray = get_current_position()
    r = posArray[2,3]
    theta = posArray[0,3]
    phi = posArray[1,3]
    
    x=r*np.cos(theta)* np.sin(phi)*100
    y= r*np.sin(theta)* np.sin(phi)*100
    z= r*np.cos(theta)*100
    
    ra = np.sqrt((x*x) + y*y)
    #print("Polar:\n")
    #print([r, theta, phi])
    #print("cart:\n")
    print([x,y])
    print(ra)
    
    sleep(1)

## Mobile io
- constants
    - mobile (node group)
    - mobile_fbk (group feedback)
- side effects
    - logging to /logs

In [ ]:

from time import time
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
from scipy.spatial.transform import Rotation as R
# Wait 2 seconds for the module list to populate
sleep(2.0)

family_name = "HEBI"
module_name = "Mobile IO"

mobile = lookup.get_group_from_names([family_name], [module_name])

if mobile is None:
    print('Group not found: Did you forget to set the module family and name above?')
    exit(1)

# Live Visualization
# Starts logging in the background. Note that logging can be enabled at any time, and that it does not negatively
# affect the performance of your running programs.
mobile.start_log('dir', 'logs', mkdirs=True)


mobile_fbk = hebi.GroupFeedback(mobile.size)  
print(mobile, mobile_fbk) 

In [ ]:
def get_mobile_pose():
    global mobile_fbk
    mobile_fbk = mobile.get_next_feedback(reuse_fbk=mobile_fbk)
    if mobile_fbk is None:
        return "Could not get feedback"
    orient = mobile_fbk[0].ar_orientation
    pos = mobile_fbk[0].ar_position
    return (pos, orient)